In [2]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("assignment4").config("spark.some.config.option","some-value").getOrCreate()

### Question 1

In [20]:
from pyspark.sql.functions import hour, count
df = spark.read.format("csv").option("header", "true").load("BreadBasket_DMS.csv")
df = df.filter(hour(df["Time"]) >= 9).filter(hour(df["Time"]) < 23)
items = df.groupBy("Item").agg(count("*").alias("count")).orderBy("count", ascending=False).limit(5)
items.show()

+------+-----+
|  Item|count|
+------+-----+
|Coffee| 5259|
| Bread| 3151|
|   Tea| 1414|
|  Cake| 1017|
|Pastry|  797|
+------+-----+



### Question 2

In [21]:
df = spark.read.format("csv").option("delimiter", ";").option("header", "true").load("Restaurants_in_Durham_County_NC.csv")

counts = df.groupBy("rpt_area_desc").count().orderBy("count", ascending=True)

counts.show()

+--------------------+-----+
|       rpt_area_desc|count|
+--------------------+-----+
|   Bed&Breakfast Inn|    2|
|   Local Confinement|    2|
|  Bed&Breakfast Home|    4|
|        Summer Camps|    4|
|      Adult Day Care|    5|
|                null|   13|
|        Institutions|   30|
|Tattoo Establishm...|   32|
|             Lodging|   62|
|    School Buildings|   89|
|         Mobile Food|  147|
|    Residential Care|  154|
|            Day Care|  173|
|         Summer Food|  242|
|      Swimming Pools|  420|
|        Food Service| 1093|
+--------------------+-----+



### Question 3

In [3]:
from pyspark.sql.functions import lag, col, when, round
from pyspark.sql.functions import when
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col
from pyspark.sql.functions import expr

df = spark.read.format('csv').options(inferSchema='true', header='true').load('populationbycountry19802010millions.csv')
df = df.replace(['NA', '--'], [None, None])

df = df.fillna(0)

df_melted = df.selectExpr(
    "_c0",
    "stack(31, '1980', cast(`1980` as string), '1981', cast(`1981` as string), '1982', cast(`1982` as string), '1983', cast(`1983` as string), '1984', cast(`1984` as string), '1985', cast(`1985` as string), '1986', cast(`1986` as string), '1987', cast(`1987` as string), '1988', cast(`1988` as string), '1989', cast(`1989` as string), '1990', cast(`1990` as string), '1991', cast(`1991` as string), '1992', cast(`1992` as string), '1993', cast(`1993` as string), '1994', cast(`1994` as string), '1995', cast(`1995` as string), '1996', cast(`1996` as string), '1997', cast(`1997` as string), '1998', cast(`1998` as string), '1999', cast(`1999` as string), '2000', cast(`2000` as string), '2001', cast(`2001` as string), '2002', cast(`2002` as string), '2003', cast(`2003` as string), '2004', cast(`2004` as string), '2005', cast(`2005` as string), '2006', cast(`2006` as string), '2007', cast(`2007` as string), '2008', cast(`2008` as string), '2009', cast(`2009` as string), '2010', cast(`2010` as string))"
).withColumnRenamed("col0", "Year").withColumnRenamed("col1", "Population")

df_melted = df_melted.withColumn("Population", when((col("Population") == "NA") | (col("Population") == "--"), 0).otherwise(col("Population")))

df = df_melted
w = Window.partitionBy("_c0").orderBy("Year")

df = df.withColumn("prev_pop", lag("Population", 1).over(w))
df = df.withColumn("yearly_increase", col("Population") - col("prev_pop"))


df = df.withColumn("percent_increase", when(col("prev_pop").isNull(), 0)
                   .otherwise(col("yearly_increase") / col("prev_pop") * 100))

df = df.withColumn("percent_increase", when((col("percent_increase") == None),0).otherwise(col("percent_increase")))

df1 = df.alias("df1")
df2 = df.alias("df2")

result = df1.join(df2, 'Year')

result = result.select('Year', 'df1._c0', 'df1.percent_increase')

result = result.filter(col('df1.percent_increase').isNotNull())

for year in result.select('Year').distinct().collect():
    year = year.Year
    print(f"\nYear {year}:")
    result.filter(col('Year') == year).orderBy('percent_increase', ascending=False).show(1)
    result.filter(col('Year') == year).orderBy('percent_increase').show(1)



Year 1987:
+----+-------------+------------------+
|Year|          _c0|  percent_increase|
+----+-------------+------------------+
|1987|French Guiana|11.111111111111121|
+----+-------------+------------------+
only showing top 1 row

+----+------------+-------------------+
|Year|         _c0|   percent_increase|
+----+------------+-------------------+
|1987|Saint Helena|-21.299638989169676|
+----+------------+-------------------+
only showing top 1 row


Year 1988:
+----+--------------+------------------+
|Year|           _c0|  percent_increase|
+----+--------------+------------------+
|1988|Cayman Islands|11.010421386497516|
+----+--------------+------------------+
only showing top 1 row

+----+----------+-------------------+
|Year|       _c0|   percent_increase|
+----+----------+-------------------+
|1988|Mozambique|-2.8836318375165324|
+----+----------+-------------------+
only showing top 1 row


Year 1984:
+----+-----+------------------+
|Year|  _c0|  percent_increase|
+----+---

+----+------+-----------------+
|Year|   _c0| percent_increase|
+----+------+-----------------+
|1996|Rwanda|19.61417728550077|
+----+------+-----------------+
only showing top 1 row

+----+----------+-------------------+
|Year|       _c0|   percent_increase|
+----+----------+-------------------+
|1996|Montserrat|-22.590068159688407|
+----+----------+-------------------+
only showing top 1 row


Year 1983:
+----+-------------+------------------+
|Year|          _c0|  percent_increase|
+----+-------------+------------------+
|1983|French Guiana|14.285714285714276|
+----+-------------+------------------+
only showing top 1 row

+----+-------------------+-------------------+
|Year|                _c0|   percent_increase|
+----+-------------------+-------------------+
|1983|Antigua and Barbuda|-3.5141890898397343|
+----+-------------------+-------------------+
only showing top 1 row


Year 1980:
+----+-----------+----------------+
|Year|        _c0|percent_increase|
+----+-----------+-----

### Question 4

In [18]:
import re

lines = spark.read.text("romeo-juliet-pg1777.txt").rdd.map(lambda r: r[0])
words = lines.flatMap(lambda line: re.sub(r'[^\w\s]','',line).lower().split())
clean_words = words.filter(lambda word: bool(re.match(r'^[0-9a-zA-Z]+$', word)))
word_counts = clean_words.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
for word, count in word_counts.takeOrdered(20, key=lambda x: -x[1]):
    print(f"{word}: {count}")

and: 773
the: 748
i: 585
to: 580
a: 483
of: 474
is: 383
that: 365
my: 360
in: 328
you: 323
thou: 277
not: 275
with: 268
for: 267
me: 265
this: 258
it: 237
be: 234
but: 186
